# How and Why does Self Attention Work? 

## Stacking Non-Linear Transformations

The prevailing paradigm in machine learning is to repeatedly perform the same non-linear transformation, initially on the input data, and then on the outputs successively. Each transformation is called a layer. "Deep" architectures are so called because they have many layers -- for example, the GPT-3 model trained at OpenAI had 96 layers in total. 

Many of the key breakthroughs in recent years have focused on resolving the problems that crop up while training deep architectures. I have covered a few of these in previous posts, such as [Batch Normalization](...) and [Dropout](...).

The earliest deep-learning architecture was what is now called a Feed-Forward Network, which in its current mature formulation, consists of a linear operation followed by a non-linear activation function $(\sigma)$ such as ReLU (Rectified Linear Unit).

$$
f(\bold{x}) = \sigma(\bold{Wx} + b)
$$


<img src="nn.svg" alt="Feed Forward Network" style="width:500px;"/>

Another popular architecture used primarily in computer vision is the Convolutional Neural Network (CNN), whose basic transformation is the convolution followed by an activation function. The idea here is that the model can learn to detect features in an image by performing non-linear transformations on small patches of the image. Then, just like the basic Feed-Forward network, the same operation can be performed on the outputs successively.


<img src="cnn.svg" alt="Feed Forward Network" style="width:800px;"/>

I won't go into to much detail on these here.


## A New Transformation

The idea of self-attention developed out of the sequence-to-sequence model architectures which were used primarily for machine translation. Here, the goal was to learn a single representation for an input sentence ("encode"), then use it to generate a translation ("decode").

It however proved difficult to compress the information needed to translate a long sentence into a single representation. 

Bahadanau et. al. (2014) introduced the concept of attention -- the model could learn to use parts of the input sentence ("attend") directly while decoding rather than rely solely on the learnt representation. Parikh et al. (2016) realized that the attention operation itself could be used for NLP tasks such as entailment. Lin et al. (2017) introduced the concept of self-attention to perform a variety of NLP tasks. 

In my earlier post on [Nadaraya-Watson Regression](...), we saw how this classic non-parametric technique can be interpreted as an early form of attention. 

Vaswani el al. (2017) realized that the self-attention mechanism could be used as the basic non-linear transformation for sequences of variable length. This also allowed the model to process tokens in parallel by incorporating positional information (check out my post on positional embeddings [here](...)). By avoiding having to explicitly process each token in sequence, it became possible to train much deeper networks. 




## So What Exactly is Self Attention?



## Let's See If it Works

Let's train a sentiment classifier. For our dataset, we will be using Yelp review dataset with GloVE embeddings. Our baseline model is a simple feedforward network that uses an average of the word embeddings. The candidate model will use an additional self-attention layer. 



### Let's start by loading the dataset

This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment .



In [46]:
import pandas as pd

dataset_filepath = "../datasets/kaggle-sentiment140/training.1600000.processed.noemoticon.csv"
pd.set_option('display.max_colwidth', None)

column_names = ["label", "id", "date", "flag", "user", "text"]
dtype_dict = {
    "label": 'int16',
    "id": "int64",
    "flag": 'string',
    "user": 'string',
    "text": 'string'
}

df = pd.read_csv(dataset_filepath, encoding='ISO-8859-1', dtype=dtype_dict, names=column_names, parse_dates=['date'])

# Replace the 
df["target"] = df["targer"].replace({4:1})

/var/folders/x4/85_9sn3d1ng9q48ff6t3fw340000gn/T/ipykernel_11940/503260375.py:15: FutureWarning: Parsed string "Mon Apr 06 22:19:45 PDT 2009" included an un-recognized timezone "PDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df = pd.read_csv(dataset_filepath, encoding='ISO-8859-1', dtype=dtype_dict, names=column_names, parse_dates=['date'])


In [49]:
df["target"].unique()

array([0, 4])

Let's look at some reviews

In [2]:
from IPython.display import display, HTML

display(HTML("""
<style>
    .custom-paragraph {
        width: 600px;
        margin: auto;
        line-height: 1.6;
    }
</style>
"""))

In [3]:
n_samples = 5
#samples = list(islice(train_data, n_samples))
samples = list(raw_train_data.take(n_samples).cache())
html = ""
for d in samples:
    text = d[0].numpy().decode('utf-8')
    label = "Positive" if d[1] == 1 else "Negative"
    html += f"""<div class="custom-paragraph">
              <p><b>[{label}]</b> {text}</p>
             </div>"""
display(HTML(html))

2024-10-24 21:14:45.216187: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Let's preprocess the dataset

In [4]:
from itertools import islice
import spacy

nlp = spacy.load("en_core_web_sm")

# split the data into sentences
def get_sentences(raw_dataset):

    for example in raw_dataset: 
        label = example[1]
        text = d[0].numpy().decode('utf-8')
        sentences = text.split(".")[0]
        yield sentences, label


train_sentences = get_sentences(raw_train_data)
test_sentences = get_sentences(raw_test_data)

MIN_SENTENCE_LENGTH = float('inf')
MAX_SENTENCE_LENGTH = 0

sample_sentences = islice(raw_train_data, 2)
for e in sample_sentences:
    example, label = e[0].numpy().decode('utf-8'), e[1]
    tokens = [token.text.lower() for token in nlp(example)]
    print(tokens)

# Load each example into memory

    # Lowercase and split on whitespace

    # Add to vocabulary

    # Create token indexes

['this', 'was', 'an', 'absolutely', 'terrible', 'movie', '.', 'do', "n't", 'be', 'lured', 'in', 'by', 'christopher', 'walken', 'or', 'michael', 'ironside', '.', 'both', 'are', 'great', 'actors', ',', 'but', 'this', 'must', 'simply', 'be', 'their', 'worst', 'role', 'in', 'history', '.', 'even', 'their', 'great', 'acting', 'could', 'not', 'redeem', 'this', 'movie', "'s", 'ridiculous', 'storyline', '.', 'this', 'movie', 'is', 'an', 'early', 'nineties', 'us', 'propaganda', 'piece', '.', 'the', 'most', 'pathetic', 'scenes', 'were', 'those', 'when', 'the', 'columbian', 'rebels', 'were', 'making', 'their', 'cases', 'for', 'revolutions', '.', 'maria', 'conchita', 'alonso', 'appeared', 'phony', ',', 'and', 'her', 'pseudo', '-', 'love', 'affair', 'with', 'walken', 'was', 'nothing', 'but', 'a', 'pathetic', 'emotional', 'plug', 'in', 'a', 'movie', 'that', 'was', 'devoid', 'of', 'any', 'real', 'meaning', '.', 'i', 'am', 'disappointed', 'that', 'there', 'are', 'movies', 'like', 'this', ',', 'ruining

2024-10-24 21:14:46.762478: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Now, let's load the GloVE Embeddings



In [5]:
import numpy as np

def load_glove_embeddings(filepath):
    embeddings = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Load the 100-dimensional GloVe embeddings
glove_embeddings = load_glove_embeddings('../datasets/glove/glove.6B.100d.txt')

In [16]:
glove_embeddings[","]

array([-0.10767  ,  0.11053  ,  0.59812  , -0.54361  ,  0.67396  ,
        0.10663  ,  0.038867 ,  0.35481  ,  0.06351  , -0.094189 ,
        0.15786  , -0.81665  ,  0.14172  ,  0.21939  ,  0.58505  ,
       -0.52158  ,  0.22783  , -0.16642  , -0.68228  ,  0.3587   ,
        0.42568  ,  0.19021  ,  0.91963  ,  0.57555  ,  0.46185  ,
        0.42363  , -0.095399 , -0.42749  , -0.16567  , -0.056842 ,
       -0.29595  ,  0.26037  , -0.26606  , -0.070404 , -0.27662  ,
        0.15821  ,  0.69825  ,  0.43081  ,  0.27952  , -0.45437  ,
       -0.33801  , -0.58184  ,  0.22364  , -0.5778   , -0.26862  ,
       -0.20425  ,  0.56394  , -0.58524  , -0.14365  , -0.64218  ,
        0.0054697, -0.35248  ,  0.16162  ,  1.1796   , -0.47674  ,
       -2.7553   , -0.1321   , -0.047729 ,  1.0655   ,  1.1034   ,
       -0.2208   ,  0.18669  ,  0.13177  ,  0.15117  ,  0.7131   ,
       -0.35215  ,  0.91348  ,  0.61783  ,  0.70992  ,  0.23955  ,
       -0.14571  , -0.37859  , -0.045959 , -0.47368  ,  0.2385

Let's build the embedding matrix

In [11]:
def build_embedding_matrix(vocab, glove_embeddings, embedding_dim=100):
    embedding_matrix = np.random.normal(size=(len(vocab), embedding_dim)).astype('float32')
    for word, idx in vocab.items():
        if word in glove_embeddings:
            embedding_matrix[idx] = glove_embeddings[word]
    return embedding_matrix

# Example vocabulary
vocab = {'the': 0, 'dog': 1, 'ran': 2, 'fast': 3, '<PAD>': 4, '<UNK>': 5, "don't": 6}

# Create the embedding matrix
embedding_dim = 100
embedding_matrix = build_embedding_matrix(vocab, glove_embeddings, embedding_dim)


Let's define the baseline model

In [8]:
from flax import linen as nn
import jax.numpy as jnp
from flax.core.frozen_dict import freeze

## Calculate TF-IDF for all tokens ##
## Term Frequency


class Baseline(nn.Module):
    vocab_size: int
    embedding_dim: int

    @nn.compact
    def __call__(self, x):
        embedding_layer = nn.Embed(
            num_embeddings=self.vocab_size,
            features=self.embedding_dim,
            embedding_init=lambda shape, dtype: jnp.array(embedding_matrix)
        )
        embedded = embedding_layer(x)
        return embedded

In [9]:
import jax

# Example input batch (batch_size=2, sequence_length=3)
input_data = jnp.array([[0, 1, 2], [1, 3, 4]])

# Initialize and apply the model
model = Baseline(vocab_size=len(vocab), embedding_dim=embedding_dim)

# Since Flax models are functional, we need to initialize parameters
params = model.init(jax.random.PRNGKey(42), input_data)
output = model.apply(freeze(params), input_data)

print(output)  # Output shape: (2, 3, 100)


TypeError: Baseline.__call__.<locals>.<lambda>() takes 2 positional arguments but 3 were given

In [ ]:
list(glove_embeddings.keys())[1]

### References:

1. Bahdanau et al. 2015
2. Rocktashel et al. 2016 Reasoning about Entailment with Neural Attention ()
3. Parikh, A Decomposable Attention Model for Natural Language Inference
4. GloVe, Pennington et al. (2014)
5. Lin et al. A Structured Self-Attentive Sentence Embedding
6. GloVe Embeddings